#### Imports

In [39]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import joblib
from sklearn.preprocessing import RobustScaler

In [40]:
df = pd.read_csv('../../Datasets/final_dataset_descr.csv', sep='\t')

In [41]:
df.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,descripcion,quarter,month,close_quarter,age
0,201701,10001,20001,0,11,99.43861,99.43861,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,1,0,0
1,201701,10002,20001,0,17,38.68301,35.72806,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,1,0,0
2,201701,10003,20001,0,17,143.49426,143.49426,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,1,0,0
3,201701,10004,20001,0,9,184.72927,184.72927,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,1,0,0
4,201701,10005,20001,0,23,19.08407,19.08407,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,1,0,0


#### Paso 1: Filtrar y eliminar productos con poca historia


Completamos el dataset con 0 para los producto / cliente que no existen

In [ ]:
# product_ids = [20001, 20002, 20003, 20004, 20005, 20006, 20007, 20008, 20009, 20010, 20011, 20012]

In [ ]:
# filtered_df = df[df['product_id'].isin(product_ids)]

In [ ]:
# display(filtered_df)

In [ ]:
df['periodo'] = pd.to_datetime(df['periodo'], format='%Y%m')

df = df[df['periodo'] >= '2018-12-01']

product_info = df[['product_id', 'cat1', 'cat2', 'cat3', 'brand', 'sku_size', 'descripcion']].drop_duplicates()

min_max_periods = df.groupby(['customer_id', 'product_id'])['periodo'].agg(['min', 'max']).reset_index()

all_dfs = []

cont = 1

for _, row in min_max_periods.iterrows():
    customer_id = row['customer_id']
    product_id = row['product_id']
    min_period = row['min']
    max_period = '2019-12-01'
    all_periods = pd.date_range(min_period, max_period, freq='MS')
    
    combinations = pd.DataFrame({
        'customer_id': [customer_id] * len(all_periods),
        'product_id': [product_id] * len(all_periods),
        'periodo': all_periods
    })
    
    merged_df = pd.merge(combinations, df, on=['customer_id', 'product_id', 'periodo'], how='left')
    
    merged_df['tn'] = merged_df['tn'].fillna(0)
    
    merged_df['tn'] = merged_df['tn'].fillna(0)
    merged_df['cat1'] = merged_df['product_id'].map(product_info.set_index('product_id')['cat1'])
    merged_df['cat2'] = merged_df['product_id'].map(product_info.set_index('product_id')['cat2'])
    merged_df['cat3'] = merged_df['product_id'].map(product_info.set_index('product_id')['cat3'])
    merged_df['brand'] = merged_df['product_id'].map(product_info.set_index('product_id')['brand'])
    merged_df['sku_size'] = merged_df['product_id'].map(product_info.set_index('product_id')['sku_size'])
    merged_df['descripcion'] = merged_df['product_id'].map(product_info.set_index('product_id')['descripcion'])
    
    merged_df['quarter'] = 'Q' + merged_df['periodo'].dt.to_period('Q').astype(str).str[-1]
    merged_df['month'] = merged_df['periodo'].dt.month.astype(str).str.zfill(2)
    
    merged_df['plan_precios_cuidados'] = merged_df['plan_precios_cuidados'].fillna(0)
    merged_df['cust_request_qty'] = merged_df['cust_request_qty'].fillna(0)
    merged_df['cust_request_tn'] = merged_df['cust_request_tn'].fillna(0)
    merged_df['close_quarter'] = merged_df['close_quarter'].fillna(0)
    merged_df['age'] = merged_df['age'].fillna(0)
    merged_df['mes_inicial'] = min_period
    
    all_dfs.append(merged_df)
    
    print(f"procesado {cont} de {len(min_max_periods)}")
    cont += 1

df_full = pd.concat(all_dfs, ignore_index=True)

df_full = df_full.sort_values(by=['customer_id', 'product_id', 'periodo'])

df_full['periodo'] = df_full['periodo'].dt.strftime('%Y%m')

display(df_full)

In [23]:
df_full.to_csv('final_dataset_completo_con_ceros.csv', sep='\t')

In [42]:
df_full = pd.read_csv("final_dataset_completo_con_ceros.csv", sep='\t')

In [45]:
display(df_full)

,customer_id,product_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,descripcion,quarter,month,close_quarter,age,mes_inicial
0,10001,20001,201812,0.0,20.0,254.62373,254.62373,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q4,12,1.0,23.0,2018-12-01
1,10001,20001,201901,0.0,53.0,393.26092,386.60688,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,1,0.0,24.0,2018-12-01
2,10001,20001,201902,0.0,39.0,309.90610,309.90610,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,2,0.0,25.0,2018-12-01
3,10001,20001,201903,0.0,23.0,142.87158,130.54927,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,3,1.0,26.0,2018-12-01
4,10001,20001,201904,0.0,33.0,364.37071,364.37071,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q2,4,0.0,27.0,2018-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2040578,10618,20845,201912,0.0,0.0,0.00000,0.00000,HC,PROFESIONAL,PISOS,MUSCULO,5000,Profesional menta,Q4,12,0.0,0.0,2019-11-01
2040579,10618,20886,201911,0.0,1.0,0.01884,0.01884,HC,PROFESIONAL,Gel,MUSCULO,5000,Industrial 5L,Q4,11,0.0,4.0,2019-11-01
2040580,10618,20886,201912,0.0,0.0,0.00000,0.00000,HC,PROFESIONAL,Gel,MUSCULO,5000,Industrial 5L,Q4,12,0.0,0.0,2019-11-01
2040581,10618,20953,201911,0.0,1.0,0.01817,0.01817,HC,PROFESIONAL,PISOS,MUSCULO,5000,Profesinal pisos plastificados,Q4,11,0.0,4.0,2019-11-01


In [44]:
df_full = df_full.drop(columns=['Unnamed: 0'])

#### Paso 2: Aplicar LabelEncoder a las columnas categóricas


In [46]:
categorical_cols = ['cat1', 'cat2', 'cat3', 'brand', 'descripcion', 'quarter']

label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df_full[col] = le.fit_transform(df_full[col])
    label_encoders[col] = le

display(df_full)

,customer_id,product_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,descripcion,quarter,month,close_quarter,age,mes_inicial
0,10001,20001,201812,0.0,20.0,254.62373,254.62373,1,10,47,0,3000,384,3,12,1.0,23.0,2018-12-01
1,10001,20001,201901,0.0,53.0,393.26092,386.60688,1,10,47,0,3000,384,0,1,0.0,24.0,2018-12-01
2,10001,20001,201902,0.0,39.0,309.90610,309.90610,1,10,47,0,3000,384,0,2,0.0,25.0,2018-12-01
3,10001,20001,201903,0.0,23.0,142.87158,130.54927,1,10,47,0,3000,384,0,3,1.0,26.0,2018-12-01
4,10001,20001,201904,0.0,33.0,364.37071,364.37071,1,10,47,0,3000,384,1,4,0.0,27.0,2018-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2040578,10618,20845,201912,0.0,0.0,0.00000,0.00000,1,8,54,21,5000,260,3,12,0.0,0.0,2019-11-01
2040579,10618,20886,201911,0.0,1.0,0.01884,0.01884,1,8,31,21,5000,158,3,11,0.0,4.0,2019-11-01
2040580,10618,20886,201912,0.0,0.0,0.00000,0.00000,1,8,31,21,5000,158,3,12,0.0,0.0,2019-11-01
2040581,10618,20953,201911,0.0,1.0,0.01817,0.01817,1,8,54,21,5000,256,3,11,0.0,4.0,2019-11-01


#### Paso 4: Agrupar ventas por periodo, cat1, cat2, cat3, brand, customer_id y product_id


Aplico escalado

In [47]:
scalers = {}
scaled_data = []

for (product_id, customer_id), group in df_full.groupby(['product_id', 'customer_id']):
    group = group.copy()
    scaler = StandardScaler()
    group['tn'] = scaler.fit_transform(group[['tn']])
    
    # Guardar el scaler en el diccionario con una clave única
    key = f'{product_id}_{customer_id}'
    scalers[key] = scaler
    
    # Añadir el grupo escalado a la lista
    scaled_data.append(group)

scaled_df = pd.concat(scaled_data, ignore_index=True)

joblib.dump(scalers, 'scalers.pkl')

scaled_df.to_csv('scaled_final_dataset.csv', sep='\t', index=False)

In [48]:
df_full

,customer_id,product_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,descripcion,quarter,month,close_quarter,age,mes_inicial
0,10001,20001,201812,0.0,20.0,254.62373,254.62373,1,10,47,0,3000,384,3,12,1.0,23.0,2018-12-01
1,10001,20001,201901,0.0,53.0,393.26092,386.60688,1,10,47,0,3000,384,0,1,0.0,24.0,2018-12-01
2,10001,20001,201902,0.0,39.0,309.90610,309.90610,1,10,47,0,3000,384,0,2,0.0,25.0,2018-12-01
3,10001,20001,201903,0.0,23.0,142.87158,130.54927,1,10,47,0,3000,384,0,3,1.0,26.0,2018-12-01
4,10001,20001,201904,0.0,33.0,364.37071,364.37071,1,10,47,0,3000,384,1,4,0.0,27.0,2018-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2040578,10618,20845,201912,0.0,0.0,0.00000,0.00000,1,8,54,21,5000,260,3,12,0.0,0.0,2019-11-01
2040579,10618,20886,201911,0.0,1.0,0.01884,0.01884,1,8,31,21,5000,158,3,11,0.0,4.0,2019-11-01
2040580,10618,20886,201912,0.0,0.0,0.00000,0.00000,1,8,31,21,5000,158,3,12,0.0,0.0,2019-11-01
2040581,10618,20953,201911,0.0,1.0,0.01817,0.01817,1,8,54,21,5000,256,3,11,0.0,4.0,2019-11-01


In [49]:
display(scaled_df)

,customer_id,product_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,descripcion,quarter,month,close_quarter,age,mes_inicial
0,10001,20001,201812,0.0,20.0,254.62373,0.300570,1,10,47,0,3000,384,3,12,1.0,23.0,2018-12-01
1,10001,20001,201901,0.0,53.0,393.26092,1.379834,1,10,47,0,3000,384,0,1,0.0,24.0,2018-12-01
2,10001,20001,201902,0.0,39.0,309.90610,0.752630,1,10,47,0,3000,384,0,2,0.0,25.0,2018-12-01
3,10001,20001,201903,0.0,23.0,142.87158,-0.714023,1,10,47,0,3000,384,0,3,1.0,26.0,2018-12-01
4,10001,20001,201904,0.0,33.0,364.37071,1.198002,1,10,47,0,3000,384,1,4,0.0,27.0,2018-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2040578,10495,21276,201912,0.0,0.0,0.00000,-0.333333,2,6,18,23,140,412,3,12,0.0,0.0,2019-03-01
2040579,10550,21276,201909,0.0,1.0,0.00075,-0.238688,2,6,18,23,140,412,2,9,1.0,6.0,2019-09-01
2040580,10550,21276,201910,0.0,0.0,0.00000,-0.729144,2,6,18,23,140,412,3,10,0.0,0.0,2019-09-01
2040581,10550,21276,201911,0.0,2.0,0.00371,1.696976,2,6,18,23,140,412,3,11,0.0,8.0,2019-09-01


In [50]:
scaled_df.dtypes

customer_id                int64
product_id                 int64
periodo                    int64
plan_precios_cuidados    float64
cust_request_qty         float64
cust_request_tn          float64
tn                       float64
cat1                       int64
cat2                       int64
cat3                       int64
brand                      int64
sku_size                   int64
descripcion                int64
quarter                    int64
month                      int64
close_quarter            float64
age                      float64
mes_inicial               object
dtype: object

Agrupo y sumarizo

In [51]:
grouped_df = scaled_df.groupby(['periodo', 'cat1', 'cat2', 'cat3', 'brand', 'customer_id', 'product_id', 'quarter', 'month']).agg({
    'cust_request_qty': 'sum',
    'cust_request_tn': 'sum',
    'tn': 'sum'
}).reset_index()


In [52]:
display(grouped_df[(grouped_df["customer_id"] == 10001) & (grouped_df["product_id"] == 20001)])

,periodo,cat1,cat2,cat3,brand,customer_id,product_id,quarter,month,cust_request_qty,cust_request_tn,tn
15840,201812,1,10,47,0,10001,20001,3,12,20.0,254.62373,0.300570
80495,201901,1,10,47,0,10001,20001,0,1,53.0,393.26092,1.379834
175202,201902,1,10,47,0,10001,20001,0,2,39.0,309.90610,0.752630
294172,201903,1,10,47,0,10001,20001,0,3,23.0,142.87158,-0.714023
431929,201904,1,10,47,0,10001,20001,1,4,33.0,364.37071,1.198002
583070,201905,1,10,47,0,10001,20001,1,5,31.0,439.90647,1.815680
744214,201906,1,10,47,0,10001,20001,1,6,7.0,65.92436,-1.242479
916759,201907,1,10,47,0,10001,20001,2,7,14.0,144.78714,-0.597595
1099078,201908,1,10,47,0,10001,20001,2,8,9.0,33.63991,-1.506478
1288478,201909,1,10,47,0,10001,20001,2,9,18.0,111.51691,-0.889808


Aplico DTW para agrupar los registros (series de categorias/clientes similares)

In [53]:
from tslearn.clustering import TimeSeriesKMeans
import matplotlib.pyplot as plt

pivoted_df = grouped_df.pivot_table(index=['cat1', 'cat2', 'cat3', 'brand', 'customer_id', 'product_id'], columns='periodo', values='tn').fillna(0)

inertia = []
max_clusters = 30

for k in range(4, max_clusters + 1):
    print(f"Running K: {k}")
    model = TimeSeriesKMeans(n_clusters=k, metric="dtw", random_state=0)
    model.fit(pivoted_df.values)
    inertia.append(model.inertia_)
    display(inertia)


In [ ]:
plt.plot(range(4, max_clusters + 1), inertia, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.title('Elbow Method for Optimal k')
plt.show()

In [54]:
# Debug de los valores
max_value = pivoted_df[201812].max()
min_value = pivoted_df[201812].min()

print(f"Máximo valor de la columna 'nombre_de_la_columna': {max_value}")
print(f"Mínimo valor de la columna 'nombre_de_la_columna': {min_value}")


Máximo valor de la columna 'nombre_de_la_columna': 3.4641016151377557
Mínimo valor de la columna 'nombre_de_la_columna': -2.479356382671416


In [55]:
n_clusters = 30
model = TimeSeriesKMeans(n_clusters=n_clusters, metric="dtw", random_state=0)
cluster_labels = model.fit_predict(pivoted_df.values)

pivoted_df['cluster'] = cluster_labels

grouped_df = grouped_df.merge(pivoted_df['cluster'], left_on=['cat1', 'cat2', 'cat3', 'brand', 'customer_id', 'product_id'], right_index=True)

grouped_df.to_csv('grouped_with_30_clusters_scaled.csv', index=False)

display(grouped_df)

,periodo,cat1,cat2,cat3,brand,customer_id,product_id,quarter,month,cust_request_qty,cust_request_tn,tn,cluster
0,201812,0,0,4,22,10001,20609,3,12,6.0,0.87535,2.296914,25
1,201812,0,0,4,22,10002,20609,3,12,8.0,0.27780,1.211273,15
2,201812,0,0,4,22,10003,20609,3,12,1.0,0.27256,0.342311,13
3,201812,0,0,4,22,10004,20609,3,12,1.0,0.13628,-1.040164,27
4,201812,0,0,4,22,10005,20609,3,12,7.0,0.06290,-0.011106,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2040578,201912,3,13,82,32,10367,21222,3,12,0.0,0.00000,-0.426401,3
2040579,201912,3,13,82,32,10482,21192,3,12,0.0,0.00000,-0.377964,0
2040580,201912,3,13,82,32,10482,21222,3,12,0.0,0.00000,-0.377964,0
2040581,201912,3,13,82,32,10513,21222,3,12,0.0,0.00000,-0.577350,24


#### Paso 5: Armar un modelo LSTM


In [139]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
from keras.regularizers import l2

le_factor = 0.1

def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(128, activation='tanh', return_sequences=True, kernel_regularizer=l2(le_factor), input_shape=input_shape))
    model.add(Dropout(0.1))
    
    model.add(LSTM(256, activation='tanh', kernel_regularizer=l2(le_factor), return_sequences=True))
    model.add(Dropout(0.1))
    model.add(LSTM(512, activation='tanh', kernel_regularizer=l2(le_factor), return_sequences=True))
    model.add(Dropout(0.1))
    
    
    model.add(LSTM(512, activation='tanh', kernel_regularizer=l2(le_factor), return_sequences=True))
    model.add(Dropout(0.1))
    model.add(LSTM(512, activation='tanh', kernel_regularizer=l2(le_factor), return_sequences=True))
    model.add(Dropout(0.1))
    model.add(LSTM(256, activation='tanh', kernel_regularizer=l2(le_factor), return_sequences=True))
    model.add(Dropout(0.1))
    model.add(LSTM(128, activation='tanh', kernel_regularizer=l2(le_factor)))
    model.add(Dropout(0.1))

    model.add(Dense(128, activation='tanh', kernel_regularizer=l2(le_factor)))
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(le_factor))) 
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error')
    return model

#### Paso 6: Entrenar y predecir con el modelo LSTM para cada grupo


In [114]:
import pandas as pd
total = 0

for i in range(n_clusters):
    cluster_number = i

    # Filtrar el DataFrame por el cluster deseado
    cluster_data = grouped_df[grouped_df['cluster'] == cluster_number]

    unique_combinations = cluster_data[['customer_id', 'product_id']].drop_duplicates().shape[0]
    total += unique_combinations
    print(f"Cluster {cluster_number}:")
    print(f"  Número de registros: {len(cluster_data)}")
    print(f"  Número de combinaciones únicas 'customer_id' y 'product_id': {unique_combinations}")
    print()
    
print(f"Total {total}")

Cluster 0:
  Número de registros: 150078
  Número de combinaciones únicas 'customer_id' y 'product_id': 21329

Cluster 1:
  Número de registros: 45339
  Número de combinaciones únicas 'customer_id' y 'product_id': 3798

Cluster 2:
  Número de registros: 17991
  Número de combinaciones únicas 'customer_id' y 'product_id': 7463

Cluster 3:
  Número de registros: 76999
  Número de combinaciones únicas 'customer_id' y 'product_id': 5923

Cluster 4:
  Número de registros: 140100
  Número de combinaciones únicas 'customer_id' y 'product_id': 13456

Cluster 5:
  Número de registros: 29631
  Número de combinaciones únicas 'customer_id' y 'product_id': 2417

Cluster 6:
  Número de registros: 66404
  Número de combinaciones únicas 'customer_id' y 'product_id': 5108

Cluster 7:
  Número de registros: 19475
  Número de combinaciones únicas 'customer_id' y 'product_id': 3212

Cluster 8:
  Número de registros: 81501
  Número de combinaciones únicas 'customer_id' y 'product_id': 7180

Cluster 9:
  Nú

Calculo los pesos y los guardo en un dic para mejorar la performance

In [115]:
total_tn_dict = df[df['periodo'] == 201912].groupby('product_id')['tn'].sum().to_dict()
display(total_tn_dict)

{20001: 1504.68856,
 20002: 1087.30855,
 20003: 892.50129,
 20004: 637.90002,
 20005: 593.24443,
 20006: 417.23228,
 20007: 390.43432,
 20008: 195.36854,
 20009: 495.03574000000003,
 20010: 359.59998,
 20011: 392.3829,
 20012: 173.13004,
 20013: 318.09141,
 20014: 272.02812,
 20015: 297.27663,
 20016: 273.20202,
 20017: 216.90773,
 20018: 141.63569999999999,
 20019: 351.54708,
 20020: 266.06358,
 20021: 203.76721,
 20022: 210.8346,
 20023: 181.13277,
 20024: 270.45018,
 20025: 241.83432000000002,
 20026: 235.10419,
 20027: 155.25876,
 20028: 109.92618,
 20029: 150.64869,
 20030: 102.7572,
 20031: 139.91577,
 20032: 527.79811,
 20033: 96.76212,
 20035: 179.97912,
 20037: 63.37274,
 20038: 157.68477000000001,
 20039: 128.40394,
 20041: 113.11379,
 20042: 124.20086,
 20043: 93.77222,
 20044: 59.61747,
 20045: 149.89961,
 20046: 149.9563,
 20047: 71.49763,
 20049: 124.84836,
 20050: 117.02742,
 20051: 132.46038000000001,
 20052: 95.51068,
 20053: 146.36584,
 20054: 121.2091,
 20055: 65.857

In [140]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
import numpy as np

models = {}

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=0.00001, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=False, verbose=1)

# Preparar los datos por cluster
for cluster in range(n_clusters):
    display(f'Entrenando cluster numero: {cluster}')
    cluster_data = grouped_df[grouped_df['cluster'] == cluster].copy()
    cluster_data.sort_values(by='periodo', inplace=True)
    
    X, y = [], []
    X_weights = []

    for key, data in cluster_data.groupby(['customer_id', 'product_id']):
        series = data[['cat1', 'cat2', 'cat3', 'brand', 'quarter', 'month', 'customer_id', 'product_id', 'tn']].values
        if len(series) > 2:  # Asegurarse de que haya suficientes datos
            X.append(series[:-2])  # Todos los datos excepto los últimos 2
            y.append(series[-1, -1])
            
            product_id = key[1]
            total_tn = total_tn_dict.get(product_id, 0)
            X_weights.append(total_tn)
            
    
    #Padleft para que todos los registros tengan el mismo shape
    max_len = max(len(seq) for seq in X)
    X_padded = np.array([np.pad(seq, ((max_len - len(seq), 0), (0, 0)), mode='constant') for seq in X]).astype(np.float32)
    y = np.array(y).astype(np.float32)
    X_weights = np.array(X_weights).astype(np.float32)

    # Debug
    # print(len(X))
    # print(len(X_padded))
    # print(len(X_weights))
    # print(len(y))
    callbacks = [reduce_lr]

    if cluster in [0, 12]:
        callbacks.append(early_stopping)
    
    # Construir y entrenar el modelo
    model = build_lstm_model((X_padded.shape[1], X_padded.shape[2]))
    model.fit(X_padded, y, epochs=100, verbose=2, batch_size=200, validation_split=0.2, sample_weight=X_weights, callbacks=callbacks)
    models[cluster] = model


'Entrenando cluster numero: 0'

/Users/mmarchetta/Desktop/LaboratorioIII/my_env/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
86/86 - 69s - 804ms/step - loss: 79.2314 - val_loss: 7.2578 - learning_rate: 0.0010
Epoch 2/100
86/86 - 50s - 581ms/step - loss: 3.8821 - val_loss: 1.5016 - learning_rate: 0.0010
Epoch 3/100
86/86 - 50s - 580ms/step - loss: 1.3306 - val_loss: 0.5688 - learning_rate: 0.0010
Epoch 4/100
86/86 - 50s - 578ms/step - loss: 0.8310 - val_loss: 0.3466 - learning_rate: 0.0010
Epoch 5/100
86/86 - 50s - 582ms/step - loss: 0.6943 - val_loss: 0.2825 - learning_rate: 0.0010
Epoch 6/100
86/86 - 60s - 694ms/step - loss: 0.6450 - val_loss: 0.2537 - learning_rate: 0.0010
Epoch 7/100
86/86 - 50s - 579ms/step - loss: 0.6220 - val_loss: 0.2289 - learning_rate: 0.0010
Epoch 8/100
86/86 - 47s - 550ms/step - loss: 0.6089 - val_loss: 0.2310 - learning_rate: 0.0010
Epoch 9/100
86/86 - 47s - 542ms/step - loss: 0.6076 - val_loss: 0.2326 - learning_rate: 0.0010
Epoch 10/100
86/86 - 47s - 547ms/step - loss: 0.6026 - val_loss: 0.2107 - learning_rate: 0.0010
Epoch 11/100
86/86 - 46s - 540ms/step - loss: 0.

'Entrenando cluster numero: 1'

/Users/mmarchetta/Desktop/LaboratorioIII/my_env/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

/Users/mmarchetta/Desktop/LaboratorioIII/my_env/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
18/18 - 25s - 1s/step - loss: 279.5316 - val_loss: 185.1600 - learning_rate: 0.0010
Epoch 2/100
18/18 - 10s - 543ms/step - loss: 144.5239 - val_loss: 102.1147 - learning_rate: 0.0010
Epoch 3/100
18/18 - 10s - 538ms/step - loss: 86.7310 - val_loss: 66.6358 - learning_rate: 0.0010
Epoch 4/100
18/18 - 10s - 542ms/step - loss: 61.1332 - val_loss: 49.5257 - learning_rate: 0.0010
Epoch 5/100
18/18 - 10s - 550ms/step - loss: 48.2080 - val_loss: 39.7219 - learning_rate: 0.0010
Epoch 6/100
18/18 - 10s - 567ms/step - loss: 40.1262 - val_loss: 34.7225 - learning_rate: 0.0010
Epoch 7/100
18/18 - 10s - 561ms/step - loss: 34.8426 - val_loss: 28.4844 - learning_rate: 0.0010
Epoch 8/100
18/18 - 10s - 534ms/step - loss: 30.0448 - val_loss: 24.9125 - learning_rate: 0.0010
Epoch 9/100
18/18 - 10s - 529ms/step - loss: 26.7983 - val_loss: 22.3020 - learning_rate: 0.0010
Epoch 10/100
18/18 - 10s - 543ms/step - loss: 24.4126 - val_loss: 20.0363 - learning_rate: 0.0010
Epoch 11/100
18/18 - 11s - 5

'Entrenando cluster numero: 29'

/Users/mmarchetta/Desktop/LaboratorioIII/my_env/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
20/20 - 34s - 2s/step - loss: 324.6101 - val_loss: 201.0165 - learning_rate: 0.0010
Epoch 2/100
20/20 - 11s - 535ms/step - loss: 163.6490 - val_loss: 123.3405 - learning_rate: 0.0010
Epoch 3/100
20/20 - 11s - 535ms/step - loss: 109.7340 - val_loss: 89.3375 - learning_rate: 0.0010
Epoch 4/100
20/20 - 11s - 539ms/step - loss: 84.5882 - val_loss: 75.3083 - learning_rate: 0.0010
Epoch 5/100
20/20 - 11s - 537ms/step - loss: 69.7196 - val_loss: 61.7855 - learning_rate: 0.0010
Epoch 6/100
20/20 - 11s - 537ms/step - loss: 59.8793 - val_loss: 52.8044 - learning_rate: 0.0010
Epoch 7/100
20/20 - 11s - 538ms/step - loss: 53.4258 - val_loss: 46.3325 - learning_rate: 0.0010
Epoch 8/100
20/20 - 11s - 536ms/step - loss: 47.8227 - val_loss: 42.2316 - learning_rate: 0.0010
Epoch 9/100
20/20 - 11s - 538ms/step - loss: 42.7277 - val_loss: 37.2001 - learning_rate: 0.0010
Epoch 10/100
20/20 - 11s - 536ms/step - loss: 39.2115 - val_loss: 34.9547 - learning_rate: 0.0010
Epoch 11/100
20/20 - 11s - 

#### Paso 7: Sumarizar las predicciones


In [143]:
# scalers = joblib.load('scalers.pkl')

predictions = []

for cluster in range(n_clusters):
    if cluster not in models:
        continue

    model = models[cluster]
    cluster_data = grouped_df[grouped_df['cluster'] == cluster].copy()
    
    X_pred_data = []
    keys = []
    for key, data in cluster_data.groupby(['customer_id', 'product_id']):
        series = data[['cat1', 'cat2', 'cat3', 'brand', 'quarter', 'month', 'customer_id', 'product_id', 'tn']].values
        max_len = len(series) - 1
        X_pred = np.pad(series[1:], ((max_len - len(series[1:]), 0), (0, 0)), mode='constant').astype(np.float32)
        X_pred_data.append(X_pred)
        keys.append(key)
    
    if len(X_pred_data) == 0:
        continue
    
    max_len_pred = max(len(seq) for seq in X_pred_data)
    X_pred_padded = np.array([np.pad(seq, ((max_len_pred - len(seq), 0), (0, 0)), mode='constant') for seq in X_pred_data]).astype(np.float32)
    X_pred_padded = np.reshape(X_pred_padded, (X_pred_padded.shape[0], X_pred_padded.shape[1], X_pred_padded.shape[2]))
    
    preds = model.predict(X_pred_padded, verbose=0)
    inversed_preds_df = []
    
    for key, pred in zip(keys, preds):
        customer_id, product_id = key
        scaler_key = f'{product_id}_{customer_id}'
        scaler_tn = scalers[scaler_key]
        inverse_pred = scaler_tn.inverse_transform([pred])
        
        if inverse_pred[0][0] < 0:
            inversed_preds_df.append(0)
            predictions.append([customer_id, product_id, 0])
        else:
            inversed_preds_df.append(inverse_pred[0][0])
            predictions.append([customer_id, product_id, inverse_pred[0][0]])
    
    pred_df_temp = pd.DataFrame(inversed_preds_df, columns=['prediccion'])
    pred_df_temp.to_csv(f"predicciones_temprales_cluster_pID{cluster}.csv", index=False)
    
consilated_df_temp = pd.DataFrame(predictions, columns=['customer_id', 'product_id', 'prediccion'])
consilated_df_temp.to_csv(f"predicciones_temprales_todos_clusters.csv", index=False)

In [144]:
summarized_preds = consilated_df_temp.groupby(['product_id'])['prediccion'].sum().reset_index()

final_predictions_df = pd.DataFrame(summarized_preds, columns=['product_id', 'prediccion'])

final_predictions_df.to_csv('predicciones_finales.csv', index=False)

display(final_predictions_df)

,product_id,prediccion
0,20001,1285.889299
1,20002,1122.865283
2,20003,823.382444
3,20004,663.090058
4,20005,555.758389
...,...,...
775,21263,0.013954
776,21265,0.014951
777,21266,0.011970
778,21267,0.030644
